# About

This notebook was created from [a tutorial on `ipywidgets`](https://blog.amedama.jp/entry/ipywidgets-jupyter-ui) written in Japanese.  日本人の学生は原文を読んだほうがいいかもしれません。

Typical user interface for Plotly-based visualization is built using its sister technology called `Dash` but it is a bit complex system.  Therefore instead of diving into `Dash`, we use here simpler `ipywidgets` library.

The `ipywidgets` offers various tools to add simple UI to Jupyter notebook.  If this tool is not found on the virtual environment, it can be installed using `pip` with `pip install ipywidgets`.


# A simple example: Button

UI stands for **user interface**.  A **UI element** is a basic component of UI.  UI elements are sometimes called **widgets**.

The following example uses two widgets `widgets.Button` and `widgets.Output`.  The former is a button control and the other is a text buffer used to present the output for the use of the button.  The `description='Click me'` parameter sets the button's label.

**Button Callback function**: In response to button click, the function `on_click_callback` is called. The `with output: ...` block sets the current output context to the output widget.  The output generated within this block is directed to the output widget. Functions that response to the use of UI elements are called *callback functions* or simply *callbacks*.

The `on_click_callback` is bound to the button with `button.on_click(on_click_callback)`.  This statement assigns the `on_click_callback` function to serve as a click handler for the `button`.

Finaly, the `button` and `output` are displayed with `display(button, output)`.  Try hitting the button to figure out how the code work.

In [1]:
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

button = widgets.Button(description='Click me')
output = widgets.Output(layour={'border': '1px solid black'})

def on_click_callback(clicked_button):
    with output:
        output.clear_output()
        print(f'Clicked at {datetime.now()}')

button.on_click(on_click_callback)
display(button, output)

Button(description='Click me', style=ButtonStyle())

Output()

# Integrating multiple widgets

The following let the user to customize the message with a selector tool named `selector`.  The user can select a fruit from three choices: `Apple`, `Banana`, and `Cherry`.  Then the action of the `button` reflects the choice made with the `selector`.

The callback function is called when the `button` is hit and retrieves the current value of `selector` with `selector.value`, which is included in the output message.

In [2]:
button = widgets.Button(description='Click me')

selector = widgets.Select(options=['Apple', 'Banana', 'Cherry'])
output = widgets.Output(layour={'border': '1px solid black'})

@output.capture()
def on_click_callback(clicked_button):
    print(f'Selected item: {selector.value}')

button.on_click(on_click_callback)
display(selector, button, output)

Select(options=('Apple', 'Banana', 'Cherry'), value='Apple')

Button(description='Click me', style=ButtonStyle())

Output()

# Packaging the UI build code in a function

It is always undesireable to define global variables in Jupyter notebook.  Multiple definitions of the same variables may complicate interdepence between the code cells and sometime create an unresolvable cyclic dependencies.  The best practice is to reduce the number of global variable definitions.

The following example packages all the UI code in a function named `show_widgets`.  The code has the same structure as previous examples but are placed inside a function definition.

In [3]:
def show_widgets():
    button = widgets.Button(description='Click me')
    select = widgets.Select(options=['Apple', 'Banana', 'Cherry'])
    output = widgets.Output(layour={'border': '1px solid black'})

    @output.capture()
    def on_click_callback(clicked_button) -> None:
        print(f'Selected item: {select.value}')

    button.on_click(on_click_callback)
    display(select, button, output)

# ウィジェットを表示する
show_widgets()

Select(options=('Apple', 'Banana', 'Cherry'), value='Apple')

Button(description='Click me', style=ButtonStyle())

Output()

# Anchoring the box of UI elements in the global context

Data structures created inside a function can be garbage collected if they become inaccessible from the global context.  It may be the case that `ipywidgets` implementation binds all the UI elements exposed to the notebook environment to the global context and protects them from automated memory reclamation with GC.  I am not quite sure because this is implementation dependent.  If you doubt about the implementation, you may let the UI construction function to return all the UI elements and save them in a global variable.

In [4]:
def show_widgets():
    """ウィジェットを設定する関数"""
    button = widgets.Button(description='Click me')
    select = widgets.Select(options=['Apple', 'Banana', 'Cherry'])
    output = widgets.Output(layour={'border': '1px solid black'})

    @output.capture()
    def on_click_callback(clicked_button):
        print(f'Selected item: {select.value}')

    button.on_click(on_click_callback)
    # 一連のウィジェットを VBox にまとめて返す
    return widgets.VBox([button, select, output])

# ウィジェットを表示する
box = show_widgets()
display(box)


# Observing value changes of a UI element

The `on_value_change` callback responds to operation on the selector and displays the update information.  The `on_value_change` callback is bound to the `selector` with the `observe` method of the `selector`: i.e., `selector.observe(on_value_change)`.

The update information `change` is passed to the callback automatically.  Its a dictionary where its `old` and `new` fields contain the previous and the next values set to the `selector`.

In [5]:
def show_widgets():
    selector = widgets.Select(options=['Apple', 'Banana', 'Cherry'])
    output = widgets.Output(layour={'border': '1px solid black'})

    @output.capture()
    def on_value_change(change):
        # The callback responds to value changes only.  Other types of changes include 'options' and 'index'.
        if change['name'] == 'value':
            output.clear_output()
            # print(change)  # Uncomment to see the structure of the change object
            print(f'Value changed from {change["old"]} to {change["new"]}')

    # Binding the callback to the selector
    selector.observe(on_value_change)
    display(selector, output)

show_widgets()

Select(options=('Apple', 'Banana', 'Cherry'), value='Apple')

Output()

# Laying out the UI elements

`ipywidgets` offers a few kinds of layout boxes: `Box`, `HBox` (horizontal box), `VBox` (vertical box), and `GridBox`.  A layout box is a container of multiple UI elements and it apatially arranges the UI elements according to the type of the layout box.  For example, an `HBox` arranges its UI elements horizontally, and `VBox` vertically.

The following example, uses a `Box` layout that arranges its widgets (`slider`, `selector`, and `output`) horizontally.  When the `Box` is displayed, all of the UI elements it contains are also displayed.

The `widgets.interactive` method binds the callback function to multiple UI elements.  In the following example, `widgets.interactive(on_value_change, select_value=selector, slider_value=slider)` sets, the `on_value_change` callback respond to operations on `selector` and `slider`.  Values of the two UI elements are passed to the callback through its two parameters, `select_value` and `slider_value`.


In [6]:
def show_widgets():
    slider = widgets.IntSlider(value=50, min=1, max=100, description='slider:')
    selector = widgets.Select(options=['Apple', 'Banana', 'Cherry'])
    output = widgets.Output(layour={'border': '1px solid black'})

    @output.capture(clear_output=True)
    def on_value_change(select_value, slider_value):
        print(f'value changed: {select_value=}, {slider_value=}')

    widgets.interactive(on_value_change, select_value=selector, slider_value=slider)

    box = widgets.Box([slider, selector])
    display(box, output)
    return box

box = show_widgets()

Box(children=(IntSlider(value=50, description='slider:', min=1), Select(description='select_value', options=('…

Output()

In [7]:
def show_widgets():
    buttons = [widgets.Button(description=str(i)) for i in range(8)]
    # グリッドレイアウト
    grid_box = widgets.GridBox(buttons,
                               layout=widgets.Layout(grid_template_columns="repeat(3, 100px)"))
    
    def on_click_callback(button):
        print(button.description)

    for b in buttons: b.on_click(on_click_callback)
        
    display(grid_box)

show_widgets()

GridBox(children=(Button(description='0', style=ButtonStyle()), Button(description='1', style=ButtonStyle()), …

In [8]:
def show_widgets():
    labels = [widgets.Button(description=str(i)) for i in range(8)]
    # グリッドレイアウト
    grid_box = widgets.GridBox(labels,
                               layout=widgets.Layout(grid_template_columns="repeat(3, 100px)"))
    display(grid_box)

show_widgets()

GridBox(children=(Button(description='0', style=ButtonStyle()), Button(description='1', style=ButtonStyle()), …

In [9]:
def show_widgets(num_of_tabs=5):
    # タブ毎のウィジェット
    contents = [widgets.Label(f'This is tab {i}') for i in range(num_of_tabs)] 
    tab = widgets.Tab(children=contents)
    # タブのタイトルを設定する
    for i in range(num_of_tabs):
        tab.set_title(i, f'tab {i}')
    display(tab)

show_widgets()

In [11]:
def show_widgets():
    text = widgets.Text()
    select = widgets.Select(options=[])
    button = widgets.Button(description='Add')

    def on_click_callback(b):
        # テキストの入力を選択肢として追加する
        select.options = list(select.options) + [text.value]

    button.on_click(on_click_callback)
    display(text, button, select)

show_widgets()

Text(value='')

Button(description='Add', style=ButtonStyle())

Select(options=(), value=None)

In [12]:
def show_widgets():
    # Animation control
    play = widgets.Play(
        value=50,
        min=1,
        max=100,
        step=1,
        interval=500,  # 更新間隔 (ミリ秒)
        description="play:",
    )
    slider = widgets.IntSlider(value=50, min=1, max=100, description='slider:')
    output = widgets.Output(layour={'border': '1px solid black'})

    # ウィジェットの値を連動させる
    widgets.jslink((play, 'value'), (slider, 'value'))

    @output.capture(clear_output=True)
    def on_value_change(slider_value):
        print(f'value changed: {slider_value=}')

    widgets.interactive(on_value_change, slider_value=slider)
    display(play, slider, output)

show_widgets()

Play(value=50, description='play:', interval=500, min=1)

IntSlider(value=50, description='slider:', min=1)

Output()